<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2021</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Разбор задач: ExceptionsContextManagers</b><br/>
    <br/>
    <font>Денис Сапожников</font><br/>
</center>

# Supressor

**Глушитель исключений**  
перехватывает исключения заданых (и только заданных) типов и возвращает управление потоку. Исключение не пробрасывается дальше.

```python
with supresser(type_one, ...):
    do_smth()
```

<div class="alert alert-danger">
<b>Антипаттерн:</b> лишний if
</div>

In [1]:
@contextmanager
def supresser(*types_: Type[BaseException]) -> Iterator[None]:
    try:
        yield None
    except Exception as e:
        if type(e) in types_:
            return
        else:
            raise e

NameError: name 'contextmanager' is not defined

<div class="alert alert-danger">
<b>Антипаттерн:</b> лишний if (v2)
</div>

In [ ]:
@contextmanager
def supresser(*types_: Type[BaseException]) -> Iterator[None]:
    try:
        yield None
    except Exception as e:
        if type(e) not in types_:
            raise e

<div class="alert alert-danger">
<b>Антипаттерн:</b> лишний finally
</div>

In [ ]:
@contextmanager
def supresser(*types_: Type[BaseException]) -> Iterator[None]:
    try:
        yield
    except types_:
        pass
    finally:
        pass

<div>
<b>Каноническое решение:</b>
</div>

In [ ]:
@contextmanager
def supresser(*types_: Type[BaseException]) -> Iterator[None]:
    try:
        yield
    except types_:
        pass

# Retyper

**Переименователь исключений**  
меняет тип исключения, оставляя неизменными содержимое ошибки (атрибут args) и трейсбек. Исключение пробрасывается дальше.

```python
with retyper(type_from, type_to):
    do_smth()
```

<div class="alert alert-danger">
<b>Антипаттерн:</b> потеря traceback-а
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    """
    """
    try:
        yield None
    except type_from as e:
        raise type_to(*e.args)

<div class="alert alert-danger">
<b>Антипаттерн:</b> ловить Exception вместо нужного типа
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    try:
        yield
    except Exception as exc:
        if isinstance(exc, type_from):
            to = type_to(exc)
            to.args = exc.args
            to.__traceback__ = exc.__traceback__
            raise to
        else:
            raise exc

<div class="alert alert-danger">
<b>Антипаттерн:</b> переусложнение в инициализации type_to
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    try:
        yield None
    except type_from as e:
        if type(e) is type_from:
            new_exc: BaseException = type_to(e)  # Can be created from args
            new_exc.args = e.args
            new_exc.__traceback__ = e.__traceback__
            raise new_exc

<div class="alert alert-danger">
<b>Антипаттерн:</b> if не добавляющий логики
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    try:
        yield None
    except Exception as e:
        if type(e) == type_from:
            if len(e.args) == 1:
                raise type_to(e.args[0])  # ?
            else:
                raise type_to(*e.args)
        else:
            raise

<div class="alert alert-danger">
<b>Антипаттерн:</b> не общее решение
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    try:
        yield
    except type_from as e:
        if len(e.args) == 2:
            raise type_to(e.args[0], e.args[1])
        else:
            raise type_to(e.args[0])

<div>
<b>Каноническое решение:</b>
</div>

In [ ]:
@contextmanager
def retyper(type_from: Type[BaseException], type_to: Type[BaseException]) -> Iterator[None]:
    try:
        yield
    except type_from:
        _, value, traceback_ = sys.exc_info()
        # have to check this, mypy thinks value can be None otherwise
        if isinstance(value, type_from):
            raise type_to(*value.args).with_traceback(traceback_)
        raise

# Dumper

**Дампер исключений**  
записывает в переданный поток сообщение об ошибке и пробрасывает его дальше.
```python
with dumper(stream):
    do_smth()
```
**Уточнения**
* Нужно, чтоб dumper по умолчанию писал в `sys.stderr`, если `stream is None`.
* Чтоб лучше разобраться в исключениях, что у него за аргументы и трейсбек, читайте в exceptions
* Для извлечения информации о перехваченном исключении использовать модуль sys
* Чтоб сдампить в dumper только исключение без трейсбека, можно воспользоваться traceback.format_exception_only

<div class="alert alert-danger">
<b>Антипаттерн:</b> отсуствие печати трейса и дублирование логики внутри if-ах
</div>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    try:
        yield None
    except Exception as e:
        if stream:
            print(e, file=stream)
        else:
            print(e, file=sys.stderr)

<div class="alert alert-danger">
    <b>Антипаттерн:</b> печать с трейсбеком
</div>
<span>(возможно условия задачи не совсем прозрачные)<span>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    try:
        yield
    except Exception as e:
        if stream is None:
            stream = sys.stderr
        traceback.print_exc(file=stream)
        raise e

<div class="alert alert-danger">
<b>Антипаттерн:</b> лишний flush
</div>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    stream = stream if stream is not None else sys.stderr
    try:
        yield
    except Exception as excpt:
        stream.flush()
        stream.write(*excpt.args)
        raise excpt

<div class="alert alert-danger">
<b>Антипаттерн:</b> предположение что в finally будет ошибка и лишний парсинг строк
</div>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    if stream is None:
        stream = sys.stderr
    try:
        yield None
    finally:
        info: Any = sys.exc_info()
        exc: List[str] = traceback.format_exception_only(info[0], info[1])
        stream.write(exc[0].split(":")[1])
        raise info[1]  # <- TypeError: exceptions must derive from BaseException

<div class="alert alert-danger">
<b>Антипаттерн:</b> использование builtin имен
</div>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    if stream is None:
        stream = sys.stderr
    try:
        yield None
    except Exception:
        type: Optional[Type[BaseException]] = sys.exc_info()[0]
        # ...

<div class="alert alert-danger">
<b>Антипаттерн:</b> магические константы
</div>

In [ ]:
class dumper:
    def __init__(self, stream: tp.Any = None) -> None:
        if stream is None:
            self.stream = sys.stderr
        else:
            self.stream = stream

    def __enter__(self) -> None:
        pass

    def __exit__(self, type_name: tp.Any, value: tp.Any, traceback: tp.Any) -> None:
        if type_name is not None:
            f = self.stream
            s = str(type_name)
            r = str()
            for i in range(len(s)):
                if 7 < i < len(s) - 2:
                    r += s[i]
            f.write(r)
            f.write(': ' + str(value) + '\n')

<div>
<b>Каноническое решение:</b>
</div>

In [ ]:
@contextmanager
def dumper(stream: Optional[TextIO] = None) -> Iterator[None]:
    if stream is None:
        stream = sys.stderr
    try:
        yield
    except Exception:
        etype, value, traceback_ = sys.exc_info()
        message = ''.join(
            traceback.format_exception_only(etype, value)
        )
        stream.write(message)
        raise
